In [1]:
from openai import OpenAI

In [2]:
client=OpenAI()

In [3]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="Hello World",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

[0.004826683551073074, -0.05471544340252876, 0.04558578133583069, 0.03150046244263649, -0.028311332687735558, -0.029749566689133644, -0.03143792971968651, 0.031625524163246155, -0.014202565886080265, 0.015398488380014896, 0.006073413882404566, -0.02968703582882881, -0.031031472608447075, -0.022433331236243248, 0.030890775844454765, 0.024606315419077873, -0.056622665375471115, 0.013725759461522102, 0.019040973857045174, 0.031625524163246155, 0.04499172791838646, -0.003138321451842785, -0.014523041434586048, -0.017149383202195168, 0.022120671346783638, -0.012811229564249516, -0.03786308690905571, 0.008121334947645664, 0.013850823044776917, -0.04627363011240959, 0.028764689341187477, -0.04830591753125191, -0.011763820424675941, 0.01894717663526535, -0.0011734508443623781, -0.0008949883049353957, 0.01686798967421055, -0.006120312958955765, 0.012623634189367294, -0.02135465480387211, 0.028498928993940353, -0.025684991851449013, 0.0013473677681759, 0.05024440959095955, -0.028155002743005753,

In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('Reviews.csv')


In [5]:
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

In [6]:
df = pd.read_csv('Reviews.csv',index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)


,Time,ProductId,UserId,Score,Summary,Text,combined
Id,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...


In [8]:
import tiktoken
# subsample to 1k most recent reviews and remove samples that are too long
top_n = 1000
#df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
#df.drop("Time", axis=1, inplace=True)
encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
print(len(df))


1000


In [10]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding


In [16]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, model=embedding_model))
df.to_csv("C:\\Users\\saila\\GENAI_PROJECTS\\text_generation/fine_food_reviews_with_embeddings_1k.csv")

In [18]:
import numpy as np
from openai import OpenAI
import pandas as pd
from scipy.spatial.distance import cosine

# Load the saved embeddings
df = pd.read_csv("C:\\Users\\saila\\GENAI_PROJECTS\\text_generation/fine_food_reviews_with_embeddings_1k.csv")

# Convert the embedding column from string back to list
df['embedding'] = df['embedding'].apply(eval)

# Function to get embedding for the search query
client = OpenAI()
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Function to calculate similarity
def calculate_similarity(embedding1, embedding2):
    return 1 - cosine(embedding1, embedding2)  # Cosine similarity

# Search function
def search_reviews(query, df, top_n=5):
    query_embedding = get_embedding(query)
    
    # Calculate similarity for each review
    df['similarity'] = df['embedding'].apply(lambda x: calculate_similarity(x, query_embedding))
    
    # Sort by similarity and return top results
    results = df.sort_values('similarity', ascending=False).head(top_n)
    return results[['Summary', 'Text', 'similarity']]

# Example usage
search_results = search_reviews("delicious chocolate dessert", df)
print(search_results)

                   Summary                                               Text  \
201     Delicious cocolate  This is delicious hot chocolate. It is best at...   
29               yummmmmy!  MMMMMMMM, this was too yummy!! MUST be a choco...   
31   It even smells yummy!  Someone just gave me a gift of this Bellagio S...   
33     Smooth & delicious!  BELLAGIO is a delicious hot-chocolate.  It's s...   
411  yummy hot choc--olate  I love the flavor and the speed of this cocoa ...   

     similarity  
201    0.624412  
29     0.565519  
31     0.555642  
33     0.543102  
411    0.531418  
